# Block Cipher Operation Modes

## data and constants

for the plaintext we will use a 16-bit long number between 0x0000 and 0xFFFF
for the block size b we will use 4 so that we don't have to worry about padding for now

In [134]:
import random

plaintext = random.randint(0x0000, 0xffff)

print("plaintext is:")
print(hex(plaintext)) 
print(bin(plaintext))

key = random.randint(0x0, 0xf)

print("\nkey is:")
print(hex(key))



plaintext is:
0x9353
0b1001001101010011

key is:
0x4


## common helper functions

In [124]:
## split up a hexadecimal number into its hex digits

def hex_digits(hexval):
    """
    hexval is a hexadecimal value, eg 0xabcd
    returns a list of its digit values, from most to least siginficant, 
    eg [0xa, 0xb, 0xc, 0xd]
    """
    hexdigits = []
    print(f"hexadecimal is {hex(hexval)}")

    hexstring = hex(hexval)[2:]
    #print(f"the corresponding hexstring is {hexstring}")

    for c in hexstring:
        hexdigits.append(int(c, 16))

    return hexdigits
    



## test the function

hex_digits(plaintext)


hexadecimal is 0xb6c2


[11, 6, 12, 2]

In [121]:
##

def blocks_to_val(blocks):
    """
    blocks is a list of values each between 0x0 and 0xf
    from most significant hex digit to least significant hex digit
    returns the corresponding value of the whole number

    eg blocks = [10, 3, 1]
    return value is 0xa31
    """

    print(f"the input blocks are {blocks}")

    hexdigits = [hex(val)[2:] for val in blocks]

    print(f"the resulting hexdigits are {hexdigits}")

    hexstring = "0x" + "".join(hexdigits)

    print(f"the resulting hexstring is {hexstring}")

    hexval = int(hexstring, 16)
    print(f"the corresponding value is {hexval}")

    return hexval




## test the function
print(f"initial value is {plaintext}")
blocks_to_val(hex_digits(plaintext))


initial value is 46786
hexval to split is 0xb6c2
the corresponding hexstring is b6c2
the input blocks are [11, 6, 12, 2]
the resulting hexdigits are ['b', '6', 'c', '2']
the resulting hexstring is 0xb6c2
the corresponding value is 46786


46786

## Electronic Codebook (ECB)
In this one you break up the plaintext into blocks of length 4 and apply the key K to each block.
Susceptible to attach for large numbers of blocks or repetitive plaintext

In [111]:
#ECB functions
def ECB_encrypt(p, k):
    """
    p is a number that represents the plaintext 
    k is a key between 0x0 and 0xf
    (because the block size is assumed to be = 4 bits)
    """

    print("plaintext is:")
    print(hex(p)) 

    print("\nkey is:")
    print(hex(k))

    ## convert to list of hex digits
    inp_digits = hex_digits(p)

    print(f"digits to xor with key are {inp_digits}")
 
    ## xor each digit with the key
    outp_digits = [d^k for d in inp_digits]

    print(f"output digits are {outp_digits}")

    ## check if xoring with the key again gives back initial plaintext

    re_output = [d^k for d in outp_digits]
    print(f"output of xoring with key again is {re_output}")

    ## check the effect of modifying only one block
    inp_digits[0] = 0xa

    print(f"modified input digits are {inp_digits}")
    ## xor each digit with the key
    outp_digits = [d^k for d in inp_digits]

    print(f"modified output digits are {outp_digits}")

## test the function

ECB_encrypt(plaintext, key)

    



plaintext is:
0x4010

key is:
0xb
hexval to split is 0x4010
the corresponding hexstring is 4010
digits to xor with key are [4, 0, 1, 0]
output digits are [15, 11, 10, 11]
output of xoring with key again is [4, 0, 1, 0]
modified input digits are [10, 0, 1, 0]
modified output digits are [1, 11, 10, 11]


## Cipher Block Chaining (CBC)

the blocks are P1, P2, P3, P4... 

IV is an initialization value of same length as the blocks

C1 = (P1 XOR IV) then encrypt result with key K

C2 = (P2 XOR C1) then encrypt result with key K

C3 = (P3 COR C3) then encrypt result with key K

... etc

the effect is that changing the data in any one block will affect all the subsequent blocks



In [140]:
## here for simplicity the encryption function is just an XOR

IV = random.randint(0x0, 0xf)

def CBC_encrypt(p, k):
    """
    p is a number that represents the plaintext
    k is a key between 0x0 and 0xf
    """


    ## get the blocks as a list of values, each between 0x0 and 0xf
    blocks = hex_digits(p)
    output = []

    print(f"the blocks are {blocks}")

    previous = IV
    for block in blocks:
        c = ( (block^previous) ^ k )
        print(f"block {block} encrypted to {c}")
        output.append(c)
        previous = c

    print(f"the encrypted blocks are {output}")



## test the function
print(f"the starting value is {plaintext}")
CBC_encrypt(plaintext, key)

## test effect of modifying one block
print("\n Now test the effect of modifying least significant block:\n")
p_mod = plaintext + 1
print(f"the modified starting value is {p_mod}")
CBC_encrypt(p_mod, key)

print("\n Now test the effect of modifying more significant block:\n")
p_mod = plaintext & 0xefff
print(f"the modified starting value is {p_mod}")
CBC_encrypt(p_mod, key)


    

the starting value is 37715
hexadecimal is 0x9353
the blocks are [9, 3, 5, 3]
block 9 encrypted to 13
block 3 encrypted to 10
block 5 encrypted to 11
block 3 encrypted to 12
the encrypted blocks are [13, 10, 11, 12]

 Now test the effect of modifying least significant block:

the modified starting value is 37716
hexadecimal is 0x9354
the blocks are [9, 3, 5, 4]
block 9 encrypted to 13
block 3 encrypted to 10
block 5 encrypted to 11
block 4 encrypted to 11
the encrypted blocks are [13, 10, 11, 11]

 Now test the effect of modifying more significant block:

the modified starting value is 33619
hexadecimal is 0x8353
the blocks are [8, 3, 5, 3]
block 8 encrypted to 12
block 3 encrypted to 11
block 5 encrypted to 10
block 3 encrypted to 13
the encrypted blocks are [12, 11, 10, 13]


## Cipher Feedback (CFB)

the encryption function Enc is used to generate random bits, and the output of it is XORED with the plaintext block.

C1 = P1 XOR Enc(K, IV)

C2 = P2 XOR Enc(K, C1)

C3 = P3 XOR Enc(K, C2)

and generally
Cn = Pn XOR Enc(K, Cn-1)


The decryption uses the *same* function Enc:

P1 = C1 XOR Enc(K, IV)

P2 = C2 XOR Enc(K, C1)

Pn = Cn XOR Enc(K, Cn)

